# Urbanczik-Senn plasticity (STDP and dendritic potentials)

Tutor: Johanna Senk [<span>&#9993;</span>](mailto:j.senk@sussex.ac.uk)  
PLENA 2024

We use the the synapse model `urbanczik_synapse` for connecting suitable multicompartment neurons. In contrast to most STDP models, the synaptic weight depends on the postsynaptic dendritic potential, in addition to the pre- and postsynaptic spike timing.  
This synapse model is compatible to the neuron model `pp_cond_exp_mc_urbanczik`, a two-compartment point process neuron with conductance-based synapses.  
For details, refer to the NEST documentation: [urbanczik_synapse](https://nest-simulator.readthedocs.io/en/stable/models/urbanczik_synapse.html) and [pp_cond_exp_mc_urbanczik](https://nest-simulator.readthedocs.io/en/stable/models/pp_cond_exp_mc_urbanczik.html).

This notebook is based on the NEST example
[Weight adaptation according to the Urbanczik-Senn plasticity](https://nest-simulator.readthedocs.io/en/stable/auto_examples/urbanczik_synapse_example.html).  
An earlier version of the notebook was developed by Agnes Korcsak-Gorzo for BNNI 2022.

## Background

<img src="urbanczik2014learning_fig1a_neuron-model.png" alt="sketch" width="300" align="center"/>

*Urbanczik and Senn et al. (2014) Fig 1A*

The Urbanczik-Senn rule (Urbanczik and Senn, 2014) applies to synapses that connect to dendrites of multicompartment model neurons. The main idea of this learning rule is to adjust the weights of dendritic synapses such that the dendrite can predict the firing rate of the soma.  
The dendrite expects the firing rate to be high when the dendrite’s membrane potential is elevated due to many incoming spikes at the dendrite, and to be low if there are only a few incoming spikes. Thus, for this prediction to be true, synapses that transmit a spike toward the dendrite while the firing rate of the soma is low are depressed and those that provide input while the soma’s firing rate is high are facilitated. Learning can be triggered by applying a teacher signal to the neuron via somatic synapses such that the actual somatic firing deviates from the dendritic prediction.

We assume the following equation to be the general form for plasticity models. The change of a weight of a synapse from neuron $j$ to neuron $i$ is given by:
\begin{align}
\frac{dW_{ij}(t)}{dt} = F(W_{ij}(t), s^*_i (t), s^*_j (t), V^*_i (t))
\end{align}

The right-hand side is here:
\begin{align}
F(s^*_j, V^*_i) &=\eta \kappa * (V^*_i s^*_j)\\
\text{with } V^*_i &= (s_i - \phi (V_i)) h (V_i)\\
s^*_j &= \kappa_s * s_j
\end{align}

$\kappa$ and $\kappa_s$ are exponential filter kernels:
\begin{align}
\kappa (t) = H(t) \frac{1}{\tau} \exp(-\frac{1}{\tau})
\end{align}

$\eta$ is the learning rate, and $\phi$ and $h$ are non-linearities.

$V^*_i$ can be interpreted as a prediction error, which never vanishes as spikes $s_i$ (point process) are compared against a rate prediction $\phi(V_i)$ (continuous signal).


### References

**Plasticity mechanism**

*R. Urbanczik, W. Senn (2014): Learning by the Dendritic Prediction of Somatic Spiking. Neuron, 81, 521-528. [doi: 10.1016/j.neuron.2013.11.030](https://doi.org/10.1016/j.neuron.2013.11.030)*

**NEST implementation**

*J. Stapmanns, J. Hahne, M. Helias, M. Bolten, M. Diesmann, D. Dahmen: Event-Based Update of Synapses in Voltage-Based Learning Rules. Frontiers in Neuroinformatics, 15:609147. [doi: 10.3389/fninf.2021.609147](https://doi.org/10.3389/fninf.2021.609147)*

## This notebook

The following experiment demonstrates the learning in a compartmental neuron where the
dendritic synapses adapt their weight according to the plasticity rule described in
Urbanczik and Senn (2014). In this simple setup, a spike pattern of 200 Poisson
spike trains is repeatedly presented to a neuron that is composed of one
somatic and one dendritic compartment. At the same time, the somatic
conductances are activated to produce a time-varying matching potential.
After the learning, this signal is then reproduced by the membrane
potential of the neuron. This script produces Fig. 1B in Urbanczik and Senn (2014) but uses standard
units instead of the unitless quantities used in the paper.

<img src="urbanczik2014learning_fig1b_time-course.png" alt="sketch" width="800" align="center"/>

*Urbanczik and Senn et al. (2014) Fig. 1B*

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import nest
nest.set_verbosity('M_ERROR')

In [ ]:
# Clear memory of NEST kernel

nest.ResetKernel()

In [ ]:
# set neuron parameters

nrn_model = 'pp_cond_exp_mc_urbanczik'

phi_params = {             # parameters of rate function
    'phi_max': .15,        # maximal rate (kHz)
    'rate_slope': .5,      # called 'k' in the paper, shift and rescale
    'beta': 1. / 3.,       # slope
    'theta': -55.,         # spiking threshold (mV), shift
    }

nrn_params = {
    't_ref': 3.,           # refractory period (ms)
    'g_sp': 600.,          # soma-to-dendritic coupling conductance (nS)
    'soma': {
        'V_m': -70.,       # initial membrane potential (mV)
        'C_m': 300.,       # membrane capacitance (pF)
        'E_L': -70.,       # resting potential (mV)
        'g_L': 30.,        # leak conductance (nS)
        'E_ex': 0.,        # exc. reversal potential (mV)
        'E_in': -75.,      # inh. reversal potential (mV)
        'tau_syn_ex': 3.,  # exc. synaptic time constant (ms)
        'tau_syn_in': 3.,  # inh. synaptic time constant (ms)
    },
    'dendritic': {
        'V_m': -70.,       # initial membrane potential (mV)
        'C_m': 300.,       # membrane capacitance (pF)
        'E_L': -70.,       # resting potential (mV)
        'g_L': 30.,        # leak conductance (nS)
        'tau_syn_ex': 3.,  # exc. synaptic time constant (ms)
        'tau_syn_in': 3.,  # inh. synaptic time constant (ms)
    },
}

nrn_params.update(phi_params)

In [ ]:
# simulation parameters

n_pattern_rep = 100         # number of repetitions of the spike pattern
pattern_duration = 200.     

t_start = 2. * pattern_duration
t_end = n_pattern_rep * pattern_duration + t_start

simulation_time = t_end + 2. * pattern_duration
n_rep_total = int(np.around(simulation_time / pattern_duration))

resolution = .1
nest.resolution = resolution

In [ ]:
# set synapse parameters

rt = nest.GetDefaults(nrn_model)['receptor_types']

syn_params = {
    'receptor_type': rt['dendritic_exc'],           # receptor type
    'tau_Delta': 100.,                              # low pass filtering time constant of weight change (ms)
    'eta': .17,                                     # learning rate
    'weight': .3 * nrn_params['dendritic']['C_m'],  # initial weight (pA)
    'Wmax': 4.5 * nrn_params['dendritic']['C_m'],   # maximal (cut-off) weight (pA)
    'delay': resolution,                            # delay (ms)
}

In [ ]:
# set somatic input parameters

def calculate_g_ex(amplitude, freq, offset, t_start, t_end):
    '''
    Returns weights for the spike generator that drives the excitatory
    somatic conductance.
    '''
    g_ex = lambda t: np.piecewise(t, [(t >= t_start) & (t < t_end)],
                                  [lambda t: amplitude * np.sin(freq * t) + offset, 0.])
    return g_ex

def calculate_g_in(amplitude, t_start, t_end):
    '''
    Returns weights for the spike generator that drives the inhibitory
    somatic conductance.
    '''
    g_in = lambda t: np.piecewise(t, [(t >= t_start) & (t < t_end)],
                                  [amplitude, 0.])
    return g_in

soma_exc_inp = calculate_g_ex(
    amplitude=.016 * nrn_params['dendritic']['C_m'],
    freq=2. * np.pi*2. / pattern_duration,
    offset=.018 * nrn_params['dendritic']['C_m'],
    t_start=t_start,
    t_end=t_end
)

soma_inh_inp = calculate_g_in(
    amplitude=.06 * nrn_params['dendritic']['C_m'],
    t_start=t_start,
    t_end=t_end
)

In [ ]:
# set dendritic input parameters

n_pg = 200    # number of poisson generators
p_rate = 10.  # spike rate of poisson generators (Hz)

In [ ]:
# create poisson generators for dendritic input

pgs = nest.Create('poisson_generator', n=n_pg, params={'rate': p_rate})
prrt_nrns_pg = nest.Create('parrot_neuron', n_pg)  # intermediate helper neuron
sr = nest.Create('spike_recorder', n_pg)

In [ ]:
# connect poisson generators to spike recorders for dendritic input

nest.Connect(pgs, prrt_nrns_pg, {'rule': 'one_to_one'})
nest.Connect(prrt_nrns_pg, sr, {'rule': 'one_to_one'})

In [ ]:
# simulate and retrieve spike pattern

nest.Simulate(pattern_duration)
t_srs = [ssr.get('events', 'times') for ssr in sr]

In [ ]:
# wipe the memory again for the main simulation

nest.ResetKernel()
nest.resolution = resolution

In [ ]:
# create neurons and devices

nrn = nest.Create(nrn_model, params=nrn_params)
prrt_nrns = nest.Create('parrot_neuron', n_pg)

spike_times_soma_inp = np.arange(resolution, simulation_time, resolution)

sg_soma_exc = nest.Create('spike_generator',
                          params={'spike_times': spike_times_soma_inp,
                                  'spike_weights': soma_exc_inp(spike_times_soma_inp)})

sg_soma_inh = nest.Create('spike_generator',
                          params={'spike_times': spike_times_soma_inp,
                                  'spike_weights': soma_inh_inp(spike_times_soma_inp)})

sg_prox = nest.Create('spike_generator', n=n_pg)


mm = nest.Create('multimeter', params={
    'record_from': nest.GetDefaults(nrn_model)['recordables'],
    'interval': .1})
wr = nest.Create('weight_recorder')
sr_soma = nest.Create('spike_recorder')

In [ ]:
# create connections

nest.Connect(sg_prox, prrt_nrns, {'rule': 'one_to_one'})

nest.CopyModel('urbanczik_synapse', 'urbanczik_synapse_wr', {'weight_recorder': wr[0]})
syn_params['synapse_model'] = 'urbanczik_synapse_wr'

nest.Connect(prrt_nrns, nrn, syn_spec=syn_params)

nest.Connect(mm, nrn, syn_spec={'delay': .1})

nest.Connect(sg_soma_exc, nrn,
             syn_spec={'receptor_type': rt['soma_exc'], 'weight': 10. * resolution, 'delay': resolution})

nest.Connect(sg_soma_inh, nrn,
             syn_spec={'receptor_type': rt['soma_inh'], 'weight': 10. * resolution, 'delay': resolution})

nest.Connect(nrn, sr_soma)

In [ ]:
# simulate in intervals

for i in np.arange(n_rep_total):
    for (sg, t_sp) in zip(sg_prox, t_srs):  # set spike times of pattern for each spike generator
        nest.SetStatus(sg, {'spike_times': np.array(t_sp) + i * pattern_duration})
        
    nest.Simulate(pattern_duration)

In [ ]:
# read out devices

mm_events = mm.events
t = mm_events['times']
U = mm_events['V_m.s']
V_w = mm_events['V_m.p']

g_in = mm_events['g_in.s']
g_ex = mm_events['g_ex.s']
I_ex = mm_events['I_ex.p']
I_in = mm_events['I_in.p']

wr_events = wr.events
senders = wr_events['senders']
targets = wr_events['targets']
weights = wr_events['weights']
times = wr_events['times']

spike_times_soma = sr_soma.get('events', 'times')

In [ ]:
# derive some further dynamic variables

def calculate_U_M(g_ex, g_in, E_ex, E_in):
    '''
    Returns the matching potential as a function of the somatic conductances.
    '''
    U_M = (g_ex * E_ex + g_in * E_in) / (g_ex + g_in)
    return U_M


def calculate_V_w_star(V_w, g_sp, g_L, E_L):
    '''
    Returns the dendritic prediction of the somatic membrane potential.
    '''
    V_w_star = (g_L * E_L + g_sp * V_w) / (g_L + g_sp)
    return V_w_star


def calculate_phi(U, phi_max, rate_slope, beta, theta):
    '''
    Returns rate function of the soma.
    '''
    phi = phi_max / (1. + rate_slope * np.exp(beta * (theta - U)))
    return phi


def calculate_h(U, rate_slope, beta, theta):
    '''
    Returns derivative of the log(phi).
    '''
    h = 15. * beta / (1. + np.exp(-beta * (theta - U)) / rate_slope)
    return h

U_M = calculate_U_M(g_ex, g_in, nrn_params['soma']['E_ex'], nrn_params['soma']['E_in'])

V_w_star = calculate_V_w_star(V_w, nrn_params['g_sp'], nrn_params['dendritic']['g_L'],
                              nrn_params['dendritic']['E_L'])

phi_U = calculate_phi(U, **phi_params)
phi_U_M = calculate_phi(U_M, **phi_params)
phi_V_w = calculate_phi(V_w, **phi_params)
phi_V_w_star = calculate_phi(V_w_star, **phi_params)

h_V_w_star = calculate_h(V_w_star, phi_params['rate_slope'], phi_params['beta'],
                         phi_params['theta'])

In [ ]:
# plot dynamic variables constituting the weight update

plt.rcParams.update({
    'font.size': 16,
    'axes.spines.right' : False,
    'axes.spines.top' : False,
})

legend_params = {
    'ncol': 1,
    'numpoints': 2,
    'labelspacing': .2,
    'columnspacing': .2,
    'loc': 'upper left',
    'bbox_to_anchor': (1., 1.2),
    'handlelength': 1.5,
}
lw = 2.5

idc = np.arange(0, len(t), dtype=int)
t_sub = np.array(t)[idc]

for xlim in ((200, 700), (1000, 1500), (19000, 19500), (20300, 20800)):

    fig, axs = plt.subplots(6, 1, sharex=True, figsize=(15, 12))

    for ax in axs:
        ax.spines.right.set_visible(False)
        ax.spines.top.set_visible(False)

    # membrane potentials and matching potential
    axs[0].plot(t_sub, U_M[idc], lw=lw, label=r'$U_M$', color='r', ls='-')
    axs[0].plot(t_sub, U[idc], lw=lw, label=r'$U$', color='darkblue')
    axs[0].plot(t_sub, V_w[idc], lw=lw, label=r'$V_\mathbf{W}$', color='deepskyblue')
    axs[0].plot(t_sub, V_w_star[idc], lw=lw, label=r'$V_\mathbf{W}^\ast$', color='b', ls='--')
    axs[0].set_ylabel('potential\n(mV)')
    axs[0].legend(**legend_params)
    axs[0].set_xlim(xlim)

    # somatic conductances
    axs[1].plot(t_sub, g_in[idc], lw=lw, label=r'$g_\mathrm{inh}$', color='coral')
    axs[1].plot(t_sub, g_ex[idc], lw=lw, label=r'$g_\mathrm{exc}$', color='r')
    axs[1].set_ylabel('somatic\nconductance\n(nS)')
    axs[1].legend(**legend_params)
    axs[1].set_xlim(xlim)

    # dendritic currents
    axs[2].plot(t_sub, I_in[idc], lw=lw, label=r'$I_\mathrm{inh}$', color='coral')
    axs[2].plot(t_sub, I_ex[idc], lw=lw, label=r'$I_\mathrm{exc}$', color='r')
    axs[2].set_ylabel('dendritic\ncurrent\n(pA)')
    axs[2].legend(**legend_params)
    axs[2].set_xlim(xlim)

    # spikes soma
    axs[3].plot(spike_times_soma, np.zeros(len(spike_times_soma)), 's', color='k', markersize=2,
                label='spikes soma')
    axs[3].set_xlim(xlim)
    axs[3].set_ylabel('spikes', labelpad=30)
    axs[3].set_yticks([0.])
    axs[3].set_yticklabels([])
    axs[3].legend(**legend_params)
    
    # rates
    axs[4].plot(t_sub, phi_U[idc] - phi_V_w_star[idc], lw=lw,
                label=r'$\phi(U) - \phi(V_\mathbf{W}^\ast)$', color='r', ls='-')
    axs[4].plot(t_sub, phi_U[idc], lw=lw, label=r'$\phi(U)$', color='darkblue')
    axs[4].plot(t_sub, phi_V_w[idc], lw=lw,
             label=r'$\phi(V_\mathbf{W})$', color='deepskyblue')
    axs[4].plot(t_sub, phi_V_w_star[idc], lw=lw,
             label=r'$\phi(V_\mathbf{W}^\ast)$', color='b', ls='--')
    axs[4].legend(**legend_params)
    axs[4].set_ylabel('rate\n(spikes/s)')
    axs[4].set_xlim(xlim)
    
    # derivative of rate
    axs[5].plot(t_sub, h_V_w_star[idc], lw=lw,
             label=r'$h(V_\mathbf{W}^\ast)$', color='g')
    axs[5].set_xlim(xlim)
    axs[5].legend(**legend_params)
    axs[5].set_ylabel('derivative\nof rate')
    axs[5].set_xlabel('time (ms)')
    
    plt.show()
    print('\n\n')

In [ ]:
# plot time course of synaptic weights

colors = ['darkblue', 'deepskyblue', 'b', 'r', 'coral']
legend_params = {
    'ncol': 1,
    'numpoints': 2,
    'labelspacing': .2,
    'columnspacing': .2,
    'loc': 'upper left',
    'bbox_to_anchor': (1., 1),
    'handlelength': 1.5,
    'title': 'sender'
}

fig, ax = plt.subplots(1, 1, figsize=(15, 4))
for i, color in zip(np.arange(2, 200, 25), colors):
    index = np.intersect1d(np.where(senders == i), np.where(targets == 1))
    if not len(index) == 0:
        ax.step(times[index], weights[index], label='{}'.format(i - 2), c=color)
ax.spines.right.set_visible(False)
ax.spines.top.set_visible(False)
ax.set_xlabel('time (ms)')
ax.set_ylabel('weight (nS)')
ax.legend(**legend_params)
ax.set_xlim(0, max(times))
plt.show()

In [ ]:
times, loss = [], []
step = int(pattern_duration/resolution)

for pattern_rep_i in range(int(t_start/pattern_duration), int(t_end/pattern_duration)):
    times.append(t_start + (pattern_rep_i + 1)*pattern_duration)
    idc = range(pattern_rep_i*step, (pattern_rep_i + 1) * step)
    loss.append(np.mean((np.array(U)[idc] - np.array(U_M)[idc])**2))

fig, ax = plt.subplots(1, figsize=(10,5))
ax.plot(times, loss, c='r')
ax.set_ylabel('loss')
_ = ax.set_xlabel('time (ms)')